## 1. Data Preprocessing
#### Preprocess Datasets to extract bounding boxes from segmentation masks

Preprocess CODEBRIM dataset:

In [ ]:
import os
from bs4 import BeautifulSoup

DATA_DIR = r'F:\Accumulated_Defect_Segmentation\CODEBRIM_original_images\original_dataset\annotations'
files = os.scandir(DATA_DIR)
for file in files:

    if file.is_file():
        with open(os.path.join(DATA_DIR, file.name), 'r') as f:
            data = f.read()

        # Output File
        outfile = open(os.path.join(DATA_DIR, "YOLO_annotations", file.name[:-4]+".txt"), "w")
        crack, spalling, efflorescence, corrosion = False, False, False, False
        h, w = 0, 0

        print("Parsing: " + file.name)
        Bs_data = BeautifulSoup(data, "xml")

        img_res = Bs_data.find('size')
        h, w = int(img_res.height.string), int(img_res.width.string)

        for obj in Bs_data.find_all("object"):
            crack, spalling, efflorescence, corrosion = False, False, False, False

            xmin, ymin, xmax, ymax = int(obj.bndbox.xmin.string), int(obj.bndbox.ymin.string), int(obj.bndbox.xmax.string), int(obj.bndbox.ymax.string)
            width, height = int(xmax - xmin), int(ymax - ymin)
            x_centre, y_centre = xmin + width/2, ymin + height/2

            # Convert to Percentile
            x_centre, y_centre, width, height = x_centre/w, y_centre/h, width/w, height/h

            crack, spalling, efflorescence = bool(int(obj.Defect.Crack.string)), bool(int(obj.Defect.Spallation.string)), bool(int(obj.Defect.Efflorescence.string))

            if bool(obj.Defect.ExposedBars.int) or bool(int(obj.Defect.CorrosionStain.string)):
                corrosion = True

            # Write in outfile
            if crack:
                outfile.write("0 " + str(x_centre) + " " + str(y_centre) + " " + str(width) + " " + str(height)+"\n")
            elif spalling:
                outfile.write("1 " + str(x_centre) + " " + str(y_centre) + " " + str(width) + " " + str(height)+"\n")
            elif efflorescence:
                outfile.write("3 " + str(x_centre) + " " + str(y_centre) + " " + str(width) + " " + str(height)+"\n")
            else:
                if corrosion:
                    outfile.write("2 " + str(x_centre) + " " + str(y_centre) + " " + str(width) + " " + str(height)+"\n")

        outfile.close()

files.close()

Preprocess QuakeCity dataset

In [ ]:
import os
import numpy as np
import cv2
from skimage.measure import label, regionprops, find_contours
from skimage.morphology import dilation, closing

""" Creating a directory """


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


""" Convert a mask to border image """


def mask_to_border(mask):
    h, w = mask.shape
    border = np.zeros((h, w))

    contours = find_contours(mask)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255

    return border


""" Mask to bounding boxes """


def mask_to_bbox(mask):
    bboxes = []

    mask = mask_to_border(mask)
    lbl = label(mask)
    props = regionprops(lbl)
    for prop in props:
        x1 = prop.bbox[1]
        y1 = prop.bbox[0]

        x2 = prop.bbox[3]
        y2 = prop.bbox[2]

        bboxes.append([x1, y1, x2, y2])

    return bboxes


def parse_mask(mask):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    return mask

def plot_bbox(img, bboxes, c):
    img_h, img_w, _ = img.shape
    for bbox in bboxes:
        x = bbox[0] * img_w
        y = bbox[1] * img_h
        w = bbox[2] * img_w
        h = bbox[3] * img_h

        x1 = int(x - w / 2)
        y1 = int(y - h / 2)
        x2 = int(x + w / 2)
        y2 = int(y + h / 2)
        

        cv2.rectangle(img, (x1, y1), (x2, y2), c, 2)

    return img

def remove_small_boxes(bboxes, min_size = 50):
    temp = bboxes
    for bbox in bboxes:
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        if width < min_size or height < min_size:
            temp.remove(bbox)
    return temp

def write_outfile(bboxes, classification, outfile, w, h):
    for bbox in bboxes:
        xmin, ymin, xmax, ymax = bbox[0], bbox[1], bbox[2], bbox[3]
        width, height = int(xmax - xmin), int(ymax - ymin)
        x_centre, y_centre = xmin + (width/2), ymin + (height/2)

        # Convert to Percentile
        x_centre, y_centre, width, height = x_centre / w, y_centre / h, width / w, height / h

        outfile.write(str(classification) + " " + str(x_centre) + " " + str(y_centre) + " " + str(width) + " " + str(height) + "\n")


if __name__ == "__main__":
    """ Load the dataset """
    DATA_DIR = r'F:\Accumulated_Defect_Segmentation\UH_QuakeCity\QuakeCity\label'
    IMG_DIR = r'F:\Accumulated_Defect_Segmentation\UH_QuakeCity\QuakeCity\image'

    """ Create dir """
    create_dir(os.path.join(DATA_DIR, "YOLO_annotations"))

    files = os.scandir(os.path.join(DATA_DIR, "crack"))
    for file in files:

        print("Processing "+file.name)

        name = file.name.split(".")[0]

        # Output File
        outfile = open(os.path.join(DATA_DIR, "YOLO_annotations", name+".txt"), "w")

        """ Read image and mask """
        image = cv2.imread(os.path.join(IMG_DIR, file.name), cv2.IMREAD_COLOR)
        h, w, _ = image.shape

        crack_mask = cv2.imread(os.path.join(DATA_DIR, "crack", file.name), cv2.IMREAD_GRAYSCALE)
        rebar_mask = cv2.imread(os.path.join(DATA_DIR, "rebar", file.name), cv2.IMREAD_GRAYSCALE)
        spall_mask = cv2.imread(os.path.join(DATA_DIR, "spall", file.name), cv2.IMREAD_GRAYSCALE)

        """ Detecting Bounding Boxes """
        crack_mask = closing(dilation(dilation(dilation(dilation(dilation(closing(closing(crack_mask))))))))
        crack_bboxes = mask_to_bbox(crack_mask)
        rebar_mask = closing(dilation(dilation(dilation(dilation(dilation(closing(closing(rebar_mask))))))))
        rebar_bboxes = mask_to_bbox(rebar_mask)
        spall_mask = closing(dilation(dilation(dilation(dilation(dilation(closing(closing(spall_mask))))))))
        spall_bboxes = mask_to_bbox(spall_mask)

        """ Remove Too Small Boxes """
        crack_bboxes = remove_small_boxes(crack_bboxes)
        rebar_bboxes = remove_small_boxes(rebar_bboxes)
        spall_bboxes = remove_small_boxes(spall_bboxes)

        """ Write to File """
        write_outfile(crack_bboxes, 0, outfile, w, h)
        write_outfile(rebar_bboxes, 2, outfile, w, h)
        write_outfile(spall_bboxes, 1, outfile, w, h)

        outfile.close()

files.close()

Preprocess S2DS dataset

In [ ]:
import sys
import glob
import os
import cv2
import shutil
import numpy as np

DATASET_DIR = '/home/zaid/datasets/defect_segment/s2ds'
OUTPUT_DIR = '/home/zaid/datasets/defect_segment/s2ds_out'

# white: cracks
# red: spalling
# yellow: corrosion
# blue: efflorescence

def main(dataset_dir, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'check'), exist_ok=True)

    train = sorted(glob.glob(os.path.join(dataset_dir, "train", "*")))
    test = sorted(glob.glob(os.path.join(dataset_dir, "test", "*")))
    val = sorted(glob.glob(os.path.join(dataset_dir, "val", "*")))
    images = train + test + val
    n = len(images)

    for i,img in enumerate(images):
        if '_lab' in img:
            continue        

        print(f"Processing image {i} of {n}", end='\r')

        img_name = img.split('/')[-1]
        shutil.copy(img, os.path.join(output_dir, 'images', img_name))
        
        mask = cv2.imread(img.replace('.png', '_lab.png'), cv2.IMREAD_COLOR)
        mask_crack = cv2.inRange(mask, np.array([255, 255, 255]), np.array([255, 255, 255]))
        mask_spall = cv2.inRange(mask, np.array([0, 0, 255]), np.array([0, 0, 255]))
        mask_eff = cv2.inRange(mask, np.array([255, 255, 0]), np.array([255, 255, 0]))
        mask_corr = cv2.inRange(mask, np.array([0, 255, 255]), np.array([0, 255, 255]))        

        label_cracks = mask_to_bbox(mask_crack)
        label_spall = mask_to_bbox(mask_spall)
        label_eff = mask_to_bbox(mask_eff)
        label_corr = mask_to_bbox(mask_corr)

        with open(os.path.join(output_dir, 'labels', img_name.replace('.png', '.txt')), 'w') as f:
            for label in label_cracks:
                f.write('0 ' + ' '.join([str(x) for x in label]) + '\n')
            for label in label_spall:
                f.write('1 ' + ' '.join([str(x) for x in label]) + '\n')
            for label in label_corr:
                f.write('2 ' + ' '.join([str(x) for x in label]) + '\n')
            for label in label_eff:
                f.write('3 ' + ' '.join([str(x) for x in label]) + '\n')

        img_check = plot_bbox(cv2.imread(img, cv2.IMREAD_COLOR), label_cracks, (255, 255, 255))
        img_check = plot_bbox(img_check, label_spall, (0, 0, 255))
        img_check = plot_bbox(img_check, label_corr, (0, 255, 255))
        img_check = plot_bbox(img_check, label_eff, (255, 255, 0))

        cv2.imwrite(os.path.join(output_dir, 'check', img_name.replace('.png', '_check.png')), img_check)
    

if __name__ == "__main__":
    main(DATASET_DIR, OUTPUT_DIR)

Preprocess Zhang dataset

In [ ]:
import sys
import glob
import os
import cv2
import shutil
import numpy as np

DATASET_DIR = '/home/zaid/datasets/defect_segment/zhang_defect_segmentation'
OUTPUT_DIR = '/home/zaid/datasets/defect_segment/zhang_defect_segmentation_out'


def main(dataset_dir, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'check'), exist_ok=True)

    images = sorted(glob.glob(os.path.join(dataset_dir, "images", "*")))
    n = len(images)

    for i,img in enumerate(images):
        if '_lab' in img:
            continue        

        print(f"Processing image {i} of {n}", end='\r')

        img_name = img.split('/')[-1]
        shutil.copy(img, os.path.join(output_dir, 'images', img_name))

        img_id  = img_name.split('.')[0]                
                
        f_crack = os.path.join(dataset_dir, 'masks', img_id+'crack'+'.jpg')
        f_spall = os.path.join(dataset_dir, 'masks', img_id+'spall'+'.jpg')
        f_corr = os.path.join(dataset_dir, 'masks', img_id+'rebar'+'.jpg')
        
        if not os.path.exists(f_crack):
            label_cracks = []
        else:
            mask_crack = cv2.imread(f_crack, cv2.IMREAD_GRAYSCALE)
            label_cracks = mask_to_bbox(mask_crack)

        if not os.path.exists(f_spall):
            label_spall = []
        else:
            mask_spall = cv2.imread(f_spall, cv2.IMREAD_GRAYSCALE)
            label_spall = mask_to_bbox(mask_spall)
        
        if not os.path.exists(f_corr):
            label_corr = []
        else:
            mask_corr  = cv2.imread(f_corr, cv2.IMREAD_GRAYSCALE)
            label_corr = mask_to_bbox(mask_corr)

        with open(os.path.join(output_dir, 'labels', img_id + '.txt'), 'w') as f:
            for label in label_cracks:
                f.write('0 ' + ' '.join([str(x) for x in label]) + '\n')
            for label in label_spall:
                f.write('1 ' + ' '.join([str(x) for x in label]) + '\n')
            for label in label_corr:
                f.write('2 ' + ' '.join([str(x) for x in label]) + '\n')


        img_check = plot_bbox(cv2.imread(img, cv2.IMREAD_COLOR), label_cracks, (255, 255, 255))
        img_check = plot_bbox(img_check, label_spall, (0, 0, 255))
        img_check = plot_bbox(img_check, label_corr, (0, 255, 255))

        cv2.imwrite(os.path.join(output_dir, 'check', img_name.replace('.png', '_check.png')), img_check)
    

if __name__ == "__main__":
    main(DATASET_DIR, OUTPUT_DIR)

## 2. Models Training
#### Train YOLO V8 Models using RoboFlow and ultralytics on each dataset

CODEBRIM_baseline.py

In [ ]:
import os
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

HOME = os.getcwd()
print(HOME)

ultralytics.checks()

rf = Roboflow(api_key="YOUR API KEY")
os.chdir('./datasets')
project = rf.workspace("cvisslab").project("codebrim-poidd")
dataset = project.version(2155).download("yolov8")


QuakeCity_baseline.py

In [ ]:
import os
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

HOME = os.getcwd()
print(HOME)

ultralytics.checks()

rf = Roboflow(api_key="YOUR API KEY")
os.chdir('./datasets')
project = rf.workspace("cvisslab").project("quakecity")
dataset = project.version(1).download("yolov8")

S2DS_baseline.py

In [ ]:
import os
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

HOME = os.getcwd()
print(HOME)

ultralytics.checks()

rf = Roboflow(api_key="YOUR API KEY")
os.chdir('./datasets')
project = rf.workspace("cvisslab").project("s2ds")
dataset = project.version(1).download("yolov8")

Zhang.py

In [ ]:
import os
import ultralytics
from ultralytics import YOLO
from roboflow import Roboflow

HOME = os.getcwd()
print(HOME)

ultralytics.checks()

rf = Roboflow(api_key="YOUR API KEY")
os.chdir('./datasets')
project = rf.workspace("cvisslab").project("zhang-3seb8")
dataset = project.version(1).download("yolov8")

DockerFile

In [ ]:
FROM nvidia/cuda:11.6.2-devel-ubuntu20.04
ARG DEBIAN_FRONTEND=noninteractive
ENV TZ=America/New_York

RUN apt-get update
RUN apt-get install -y tar git curl nano wget dialog net-tools build-essential
RUN apt-get update && apt-get install ffmpeg libsm6 libxext6  -y


RUN apt-get install -y python3.8
RUN ln -s /usr/bin/python3.8 /usr/bin/python
RUN apt-get install -y python3-pip
#RUN apt-get install -y python-dev python-setuptools

RUN python --version 

RUN mkdir projects
WORKDIR /projects/

RUN mkdir /projects/datasets

RUN pip install ultralytics==8.0.20
RUN pip install roboflow

RUN apt install -y vim

COPY ./*.py /projects/
ADD ./results /projects/results

Run Docker Container

In [ ]:
docker run --shm-size 15gb -it -v ./results:/projects/results msci623yolov8:latest

Run Training

In [ ]:
yolo task=detect mode=train model=yolov8s.pt data=./datasets/CODEBRIM-2155/data.yaml epochs=50 imgsz=1024 batch=-1 patience=10 device=0 plots=True
yolo task=detect mode=train model=yolov8s.pt data=./datasets/S2DS-2155/data.yaml epochs=50 imgsz=1024 batch=-1 patience=10 device=0 plots=True
yolo task=detect mode=train model=yolov8s.pt data=./datasets/QuakeCity-2155/data.yaml epochs=50 imgsz=1024 batch=-1 patience=10 device=0 plots=True
yolo task=detect mode=train model=yolov8s.pt data=./datasets/Zhang-2155/data.yaml epochs=50 imgsz=1024 batch=-1 patience=10 device=0 plots=True

## 3. Model Validation

#### Validate baseline models and run cross-validation between each model and each dataset

QuakeCity

In [ ]:
yolo task=detect mode=val model=./results/QuakeCity_baseline/detect/trainn/weights/best.pt data=./datasets/Zhang-1/data.yaml
yolo task=detect mode=val model=./results/QuakeCity_baseline/detect/trainn/weights/best.pt data=./datasets/S2DS-1/data.yaml
yolo task=detect mode=val model=./results/QuakeCity_baseline/detect/trainn/weights/best.pt data=./datasets/CODEBRIM-1/data.yaml


Zhang

In [ ]:
yolo task=detect mode=val model=./results/Zhang_baseline/detect/trainn/weights/best.pt data=./datasets/QuakeCity-1/data.yaml
yolo task=detect mode=val model=./results/Zhang_baseline/detect/trainn/weights/best.pt data=./datasets/S2DS-1/data.yaml
yolo task=detect mode=val model=./results/Zhang_baseline/detect/trainn/weights/best.pt data=./datasets/CODEBRIM-1/data.yaml

CODEBRIM

In [ ]:
yolo task=detect mode=val model=./results/CODEBRIM_baseline/detect/trainn/weights/best.pt data=./datasets/QuakeCity-1/data.yaml
yolo task=detect mode=val model=./results/CODEBRIM_baseline/detect/trainn/weights/best.pt data=./datasets/S2DS-1/data.yaml
yolo task=detect mode=val model=./results/CODEBRIM_baseline/detect/trainn/weights/best.pt data=./datasets/Zhang-1/data.yaml

S2DS

In [ ]:
yolo task=detect mode=val model=./results/S2DS_baseline/detect/trainn/weights/best.pt data=./datasets/QuakeCity-1/data.yaml
yolo task=detect mode=val model=./results/S2DS_baseline/detect/trainn/weights/best.pt data=./datasets/CODEBRIM-1/data.yaml
yolo task=detect mode=val model=./results/S2DS_baseline/detect/trainn/weights/best.pt data=./datasets/Zhang-1/data.yaml

#### Results

* CODEBRIM_baseline (model=yolov8x.pt)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        213       1106      0.451      0.329      0.295      0.144
       corrosion-rebar        213         61      0.334      0.262       0.18     0.0735
                 crack        213        559       0.44      0.252      0.264      0.113
         efflorescence        213        104      0.442       0.25      0.215      0.108
              spalling        213        382      0.587      0.552      0.521      0.281
```

* CODEBRIM_baseline (model=yolov8l.pt)
```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        213       1106      0.426      0.386      0.308      0.151
       corrosion-rebar        213         61      0.387      0.361      0.264      0.131
                 crack        213        559      0.343      0.342      0.255      0.106
         efflorescence        213        104      0.425       0.25       0.18     0.0931
              spalling        213        382      0.549      0.592      0.533      0.274
```

* CODEBRIM_baseline (model=yolov8m.pt)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        213       1106      0.391      0.365      0.305      0.148
       corrosion-rebar        213         61      0.333      0.377       0.24       0.12
                 crack        213        559      0.353      0.293      0.242        0.1
         efflorescence        213        104      0.403       0.24      0.222     0.0974
              spalling        213        382      0.477       0.55      0.515      0.274

```

* CODEBRIM_baseline(model=yolov8n.pt)
```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        213       1106      0.298      0.336      0.241      0.108
       corrosion-rebar        213         61      0.229      0.258      0.128     0.0539
                 crack        213        559      0.307      0.324       0.23      0.093
         efflorescence        213        104      0.236      0.212      0.149      0.062
              spalling        213        382       0.42      0.552      0.458      0.223
```

* Zhang_baseline (model=yolov8n.pt)

```
                   all        289        664      0.688      0.653      0.683      0.428
       corrosion-rebar        289        255      0.611      0.547      0.557      0.253
                 crack        289        195      0.663      0.616      0.654      0.373
              spalling        289        214      0.789      0.794      0.836      0.657
```

* Zhang_baseline (model=yolov8m.pt) (train 2)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        289        664      0.716      0.697      0.724      0.492
       corrosion-rebar        289        255      0.608      0.631      0.609      0.318
                 crack        289        195      0.727      0.605      0.682      0.426
              spalling        289        214      0.813      0.855      0.882      0.732
```

* Zhang_baseline (model=yolov8x.pt) (train 3)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        289        664      0.723      0.718      0.732      0.502
       corrosion-rebar        289        255       0.61      0.626      0.593      0.304
                 crack        289        195      0.739      0.672      0.721      0.477
              spalling        289        214      0.821      0.854      0.881      0.724
```


* S2DS_baseline (model=yolov8n.pt)

```
                   all        289        664      0.688      0.653      0.683      0.428
       corrosion-rebar        289        255      0.611      0.547      0.557      0.253
                 crack        289        195      0.663      0.616      0.654      0.373
              spalling        289        214      0.789      0.794      0.836      0.657

```

* S2DS_baseline (model=yolov8m.pt)
```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        122        892      0.578      0.395      0.422      0.254
       corrosion-rebar        122        320      0.594      0.494      0.524      0.313
                 crack        122        191      0.432      0.382      0.315      0.171
         efflorescence        122        165      0.668      0.279      0.363      0.204
              spalling        122        216      0.618      0.426      0.488      0.329

```

* S2DS_baseline (model=yolov8x.pt)
```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|████████
                   all        122        892      0.512      0.415      0.413       0.25
       corrosion-rebar        122        320      0.574      0.455      0.458      0.284
                 crack        122        191      0.411      0.482      0.365      0.208
         efflorescence        122        165      0.525      0.261      0.337      0.192
              spalling        122        216      0.538      0.463      0.494      0.317
```

* QuakeCity_baseline (model=yolov8n.pt) (train 3)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/4
                   all        962      76803      0.488      0.359      0.364      0.248
       corrosion-rebar        962        848      0.451       0.12      0.167     0.0806
                 crack        962      49759       0.47      0.331      0.336      0.217
              spalling        962      26196      0.543      0.626       0.59      0.446

```

* QuakeCity_baseline (model=yolov8m.pt) (train 9)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|████████
                   all        962      76803      0.512      0.422      0.419      0.298
       corrosion-rebar        962        848      0.471      0.192      0.232      0.122
                 crack        962      49759      0.508      0.401      0.401      0.279
              spalling        962      26196      0.558      0.672      0.624      0.493
```

* QuakeCity_baseline (model=yolov8x.pt) (train 10)

```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|████████
                   all        962      76803      0.505      0.475      0.458       0.33
       corrosion-rebar        962        848       0.45      0.273      0.295      0.154
                 crack        962      49759       0.51      0.448      0.435      0.314
              spalling        962      26196      0.556      0.703      0.643       0.52

```

#### Dataset Cross-Validation


* QuakeCity (train) ==> Zhang (test)
    * yolo_nano
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 
                   all        289        664      0.456     0.0628     0.0519     0.0289
       corrosion-rebar        289        255          1          0          0          0
                 crack        289        195     0.0473     0.0154    0.00447    0.00213
              spalling        289        214       0.32      0.173      0.151     0.0845
    ```
    * yolo_medium
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 
                   all        289        664      0.501       0.11     0.0946     0.0409
       corrosion-rebar        289        255          1          0          0          0
                 crack        289        195        0.1      0.041     0.0107    0.00315
              spalling        289        214      0.403       0.29      0.273       0.12
    ```
    * yolo_xl
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 
                   all        289        664      0.493     0.0869     0.0661     0.0316
       corrosion-rebar        289        255          1          0          0          0
                 crack        289        195     0.0717      0.041    0.00863    0.00246
              spalling        289        214      0.406       0.22       0.19     0.0923
    ```


* Zhang (train) ==> QuakeCity (test)
    * yolo_nano
    ```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [01:03<00:00,  1.04s/it]
                   all        962      76803     0.0716     0.0389     0.0497     0.0226
       corrosion-rebar        962        848   0.000113    0.00472   5.77e-05   1.73e-05
                 crack        962      49759     0.0934    0.00474     0.0514     0.0265
              spalling        962      26196      0.121      0.107     0.0977     0.0413
    ```
    * yolo_medium
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [04:22<00:00,  4.30s/it]
                   all        962      76803     0.0585     0.0606     0.0477     0.0188
       corrosion-rebar        962        848   0.000287     0.0212   0.000149   4.96e-05
                 crack        962      49759     0.0702     0.0117     0.0403     0.0196
              spalling        962      26196      0.105      0.149      0.103     0.0369
    ```
    * yolo_xl
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        962      76803     0.0576      0.069     0.0469     0.0181
       corrosion-rebar        962        848   0.000499      0.033   0.000263   7.03e-05
                 crack        962      49759     0.0749     0.0153      0.043     0.0199
              spalling        962      26196     0.0975      0.159     0.0975     0.0344
    ```

* CODEBRIM (train) ==> S2DS (test)
    * yolo_nano
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        122        892      0.223       0.11     0.0744     0.0261
       corrosion-rebar        122        320      0.255    0.00937     0.0252    0.00678
                 crack        122        191      0.175      0.173     0.0851      0.027
         efflorescence        122        165      0.213     0.0545     0.0379     0.0141
              spalling        122        216      0.249      0.204       0.15     0.0566
    ```
    * yolo_m
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████
                   all        122        892      0.175      0.137     0.0923     0.0324
       corrosion-rebar        122        320      0.175     0.0156     0.0421     0.0139
                 crack        122        191      0.121      0.236        0.1     0.0309
         efflorescence        122        165       0.14     0.0606      0.047      0.018
              spalling        122        216      0.263      0.236       0.18     0.0667
    ```
    * yolo_xl
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [01:18<00:00,  9.77s/it]
                   all        122        892      0.251      0.141     0.0934     0.0373
       corrosion-rebar        122        320      0.419     0.0219       0.05      0.015
                 crack        122        191      0.163      0.251      0.115     0.0408
         efflorescence        122        165      0.124     0.0485      0.028     0.0103
              spalling        122        216      0.298      0.241      0.181     0.0831
    ```

* S2DS (train) ==> CODEBRIM (test)
    * yolo_n
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:12<00:00,  1.10it/s]
                   all        213       1106      0.102      0.112      0.047     0.0171
       corrosion-rebar        213         61     0.0131      0.115      0.016    0.00705
                 crack        213        559      0.118     0.0751     0.0357     0.0148
         efflorescence        213        104     0.0356     0.0192    0.00983    0.00577
              spalling        213        382       0.24      0.241      0.126     0.0409
    ```
    * yolo_m
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:57<00:00,  4.09s/it]
                   all        213       1106      0.103      0.124     0.0432     0.0163
       corrosion-rebar        213         61      0.017       0.18     0.0196    0.00521
                 crack        213        559     0.0912     0.0537       0.03     0.0143
         efflorescence        213        104      0.134     0.0769     0.0341     0.0141
              spalling        213        382      0.168      0.183      0.089     0.0318
    ```
    * yolo_xl
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [02:18<00:00,  9.89s/it]
                   all        213       1106     0.0811      0.108     0.0385     0.0148
       corrosion-rebar        213         61    0.00973      0.115     0.0124    0.00308
                 crack        213        559     0.0703     0.0662     0.0271     0.0123
         efflorescence        213        104     0.0852     0.0481     0.0133    0.00629
              spalling        213        382      0.159      0.202      0.101     0.0376
    ```

* S2DS (train) ==> Zhang (test)
    * yolo_n
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:
                   all        289        664      0.202       0.24      0.152     0.0728
       corrosion-rebar        289        255      0.241      0.267      0.141     0.0537
                 crack        289        195       0.11     0.0667     0.0598     0.0202
              spalling        289        214      0.256      0.388      0.255      0.145
    ```
    * yolo_m
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:
                   all        289        664      0.257       0.29      0.195      0.101
       corrosion-rebar        289        255      0.237       0.31      0.176     0.0644
                 crack        289        195      0.231      0.102      0.107     0.0358
              spalling        289        214      0.303      0.458      0.302      0.204
    ```
    * yolo_x
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:
                   all        289        664      0.249      0.284      0.199       0.12
       corrosion-rebar        289        255      0.244      0.282      0.147     0.0582
                 crack        289        195      0.192      0.108     0.0849     0.0329
              spalling        289        214      0.312      0.463      0.366      0.268
    ```

* Zhang (train) ==> S2DS (test)
    * train_n
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01
                   all        122        892      0.241      0.111     0.0953     0.0472
       corrosion-rebar        122        320       0.39      0.153      0.135     0.0645
                 crack        122        191      0.211     0.0838     0.0668     0.0309
         efflorescence        122        165          0          0          0          0
              spalling        122        216      0.364      0.208       0.18     0.0933
    ```
    * train_m
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03
                   all        122        892      0.219       0.12      0.111     0.0621
       corrosion-rebar        122        320      0.335      0.163      0.146     0.0714
                 crack        122        191      0.165     0.0995     0.0749     0.0403
         efflorescence        122        165          0          0          0          0
              spalling        122        216      0.376      0.218      0.225      0.137
    ```
    * train_x
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05
                   all        122        892      0.309      0.122      0.117     0.0627
       corrosion-rebar        122        320      0.418      0.155      0.158     0.0723
                 crack        122        191      0.389      0.126      0.104     0.0485
         efflorescence        122        165          0          0          0          0
              spalling        122        216       0.43      0.208      0.205       0.13
    ```
* S2DS (train) ==> QuakeCity (test)
    * train_n
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:08<00:00,  
                   all        962      76803     0.0165      0.034     0.0106    0.00456
       corrosion-rebar        962        848   8.18e-05     0.0142   4.17e-05   1.14e-05
                 crack        962      49759     0.0261     0.0154     0.0182    0.00878
              spalling        962      26196     0.0234     0.0725     0.0134     0.0049
    ```
    * train_m 
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:17<00:00,  3.43it/s]
                   all        962      76803     0.0193     0.0366     0.0133    0.00535
       corrosion-rebar        962        848   0.000187     0.0283   0.000101   3.41e-05
                 crack        962      49759     0.0424     0.0115     0.0263    0.00997
              spalling        962      26196     0.0152       0.07     0.0136    0.00606
    ```
    * train_x
    ```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:36<00:00,  1.65it/s]
                   all        962      76803     0.0217     0.0411     0.0129    0.00469
       corrosion-rebar        962        848   0.000162     0.0248   8.36e-05   2.37e-05
                 crack        962      49759     0.0463      0.019     0.0272    0.00993
              spalling        962      26196     0.0186     0.0796     0.0113    0.00411
    ```
* QuakeCity (train) ==> S2DS (test)
    * train_n
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.62it/s]
                   all        122        892      0.297      0.024    0.00647    0.00224
       corrosion-rebar        122        320          1          0          0          0
                 crack        122        191      0.103     0.0681     0.0195    0.00669
         efflorescence        122        165          0          0          0          0
              spalling        122        216     0.0836     0.0278    0.00634    0.00228
    ```
    * train_m
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.73it/s]
                   all        122        892      0.348     0.0347     0.0155    0.00588
       corrosion-rebar        122        320          1          0          0          0
                 crack        122        191       0.23      0.105     0.0433     0.0154
         efflorescence        122        165          0          0          0          0
              spalling        122        216      0.163     0.0342     0.0186    0.00811
    ```
    * train_x
    ```
                     Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.42it/s]
                   all        122        892       0.37     0.0271      0.015    0.00703
       corrosion-rebar        122        320          1          0          0          0
                 crack        122        191      0.221     0.0576     0.0295     0.0132
         efflorescence        122        165          0          0          0          0
              spalling        122        216      0.258     0.0509     0.0303     0.0149
    ```